In [13]:
# Importing necessary libraries
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance

# Defining the provided functions and parameters

# Define the weight function
def omega(t, u, x, c1, c2):
    sigma = math.sqrt(np.var(Y))
    h1 = (c1/math.sqrt(12))*(T ** (-(1/5)))
    h2 = c2*sigma*(T ** (-(1/5)))
    numerator = K_h1(u - t / T, h1) * np.prod(K_h2(x - Y[t-1], h2))
    denominator = np.sum([K_h1(u - s / T, h1) * np.prod(K_h2(x - Y[s-1], h2)) for s in range(1, T+1)])
    return numerator / denominator

def ecdf(u, x, v):
    weights = np.array([omega(t, u, x, c1, c2) for t in range(1, T+1)])
    indicator = (Y <= v).astype(int)
    return np.sum(weights * indicator)

# Define uniform kernels
def K_h1(z, h1):
    return 0.5 if abs(z/h1) <= 1 else 0

# Define Gaussian kernels
def K_h2(z, h2):
    return np.exp(-z**2 / (2 * h2**2))

# Parameters
T = 100  # Length of time series
d = 1  # Number of covariates
c1 = 5
c2 = 1.68
u_test = 0.5
x_test = np.array([0.5])
#v_values = np.linspace(-2, 2, T)  # Placeholder values, will be updated in the loop

# Running the replication loop
W1_list = []

for rep in range(1):
    # Generate locally stationary data
    np.random.seed(rep)  # Use different seed for each replication
    
    # Time series with noise
    for t in range(1, T):
        Y[t] = (0.2 + 0.3 * np.sin(0.1 * t/T)) * Y[t-1] + np.random.normal(0, 1, T)
    
    return Y
    
    # Plot the generated data
    plt.figure(figsize=(12, 5))
    plt.plot(Y, label="Simulated data", linewidth=1)
    plt.title(f"Simulated Locally Stationary Data T={T}")
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.legend()
    plt.grid(True)
    plt.show()
    
    v_values = np.linspace(Y.min(), Y.max(), T)  # Update v_values based on new Y
    
    # Test the weight function for a given (u, x)
    weights = [omega(t, u_test, x_test, c1, c2) for t in range(1, T+1)]
    
    # Values for plotting
    F_values = [ecdf(u_test, x_test, v) for v in v_values]
    
    # True CDF
    data_sorted = np.sort(Y)
    True_CDF = np.arange(1, len(data_sorted) + 1) / len(data_sorted)
    
    #Plot the estimated distribution function
    plt.figure(figsize=(12, 5))
    plt.step(data_sorted, True_CDF, where='post', label='True Distribution Function')
    plt.plot(v_values, F_values, label="Estimated Distribution Function", color='red')
    plt.title(f"Comparison between the True and Estimated Distribution Functions for T = {T}")
    plt.xlabel("Value")
    plt.ylabel("Probability")
    plt.legend()
    plt.grid(True)
    plt.show()

    
    # Compute the 1-Wasserstein distance
    W1 = wasserstein_distance(True_CDF, F_values)
    W1_list.append(W1)

# Computing mean 1-Wasserstein distance
mean_W1 = np.mean(W1_list)

print("1-Wasserstein Distance: ", W1_list)
print("Average 1-Wasserstein Distance: ", mean_W1)


SyntaxError: 'return' outside function (2428670371.py, line 51)